<a href="https://colab.research.google.com/github/psingh-xyz/NN-DL/blob/main/NN_DL_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,fbeta_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel("/content/Medical-Dataset.xlsx")

print(df)
#dataset= pd.read_csv("C:\Users\psingh\Downloads\Telegram Desktop\Medical-Dataset.xlsx")

       Label                                               Text
0          4  Catheterization laboratory events and hospital...
1          5  Renal abscess in children. Three cases of rena...
2          2  Hyperplastic polyps seen at sigmoidoscopy are ...
3          5  Subclavian artery to innominate vein fistula a...
4          4  Effect of local inhibition of gamma-aminobutyr...
...      ...                                                ...
14433      4  Quadricuspid aortic valve and aortic regurgita...
14434      1  Mammographic measurements before and after aug...
14435      1  Use of leukocyte-depleted platelet concentrate...
14436      2  Complications of Tenckhoff catheters post remo...
14437      3  Fatal or severely disabling cerebral infarctio...

[14438 rows x 2 columns]


Data Pre-Processing

In [ ]:
df['Label'].value_counts()

5    4805
1    3163
4    3051
3    1925
2    1494
Name: Label, dtype: int64

In [ ]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.25, stratify=df['Label'], random_state=42)

# Verify the split
print("Training set size:", len(train_df))
print("Testing set size:", len(test_df))


Training set size: 10828
Testing set size: 3610


In [ ]:
train_df.head()

,Label,Text
11519,1,Results of contemporary radical cystectomy for...
14335,5,Intraluminal pressure adjacent to left colonic...
3221,4,The Greenfield filter as the primary means of ...
7749,2,Effectiveness of periodic checkup by ultrasono...
590,4,Deep venous thrombosis. Implications after ope...


In [ ]:
train_df['Label'].value_counts()

5    3604
1    2372
4    2288
3    1444
2    1120
Name: Label, dtype: int64

In [ ]:
test_df.head()

,Label,Text
11085,5,Isolated ventral pancreatitis in a patient wit...
2306,3,Anteromedial tibial tubercle transfer without ...
10374,5,Effects of prostaglandin E1 in postoperative s...
7632,1,Carcinosarcoma of the lung with hypertrophic p...
6044,2,Scleroderma esophagus. Scleroderma (systemic s...


In [ ]:
test_df['Label'].value_counts()

5    1201
1     791
4     763
3     481
2     374
Name: Label, dtype: int64

In [ ]:
from nltk.corpus import stopwords
stop_words_list= set(stopwords.words('english'))



In [ ]:


def decode_abbreviations(Text):
    # Add your abbreviation decoding logic here
    # Define a dictionary of abbreviations and their full forms
    abbreviations = {
        'thnx': 'thanks',
        'thx': 'thanks',
        'btw': 'by the way',
        'pls': 'please',
        'plz': 'please',
        # Add more abbreviations as needed
    }

    # Replace abbreviations with their full forms
    for abbreviation, full_form in abbreviations.items():
        Text = Text.replace(abbreviation, full_form)

    return Text

# Example usage
input_text = "thnx for the info btw, pls send it ASAP."
output_text = decode_abbreviations(input_text)

print("Input text: ", input_text)
print("Output text:", output_text)




Input text:  thnx for the info btw, pls send it ASAP.
Output text: thanks for the info by the way, please send it ASAP.


In [ ]:


def preprocess_text(text):
  # Step 1: Removal of hashtags, HTML tags, mentions, URLs, and punctuations
    # Removal of hashtags, HTML tags, mentions, punctuations, and URLs
    text = re.sub(r'#\w+', '', text)             # Remove hashtags
    text = re.sub(r'<.*?>', '', text)            # Remove HTML tags
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)   # Remove mentions numbers
    text = re.sub(r'[^\w\s]', '', text)          # Remove punctuations
    text = re.sub(r'http\S+', '', text)          # Remove URLs

    # Step 2: Removal of duplicate content
    Text = df['Text'].drop_duplicates()

     # Step 3: Decoding of abbreviations
    Text = decode_abbreviations(Text)


    # Step 4: Changing to lowercase
    text = text.lower()

    # Step 5:Remove stop words
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words_list]
    filtered_text = ' '.join(filtered_words)
    return filtered_text



In [ ]:
# Apply Pre-processing
train_df['Text']= train_df['Text'].apply(lambda x: preprocess_text(x))
train_df['Text'].head()

11519    results contemporary radical cystectomy invasi...
14335    intraluminal pressure adjacent left colonic an...
3221     greenfield filter primary means therapy venous...
7749     effectiveness periodic checkup ultrasonography...
590      deep venous thrombosis implications open heart...
Name: Text, dtype: object

In [ ]:
# Apply Pre-processing
#train_df['Text']= train_df['Text'].apply(lambda x: preprocess_text(x))
#train_df['Text'].head()
test_df['Text']= test_df['Text'].apply(lambda x: preprocess_text(x))
test_df['Text'].head()

11085    isolated ventral pancreatitis patient pancreas...
2306     anteromedial tibial tubercle transfer without ...
10374    effects prostaglandin e1 postoperative surgica...
7632     carcinosarcoma lung hypertrophic pulmonary ost...
6044     scleroderma esophagus scleroderma systemic scl...
Name: Text, dtype: object

#TF-IDF weighting scheme

In [ ]:
#apply unigram method to generate feature matrix
#tv = TfidfVectorizer(ngram_range=(1,1), max_features=3000)
#train_unigra_features = tv.fit_transform(x_train).toarray()
#test_unigra_features = tv.transform(x_test).toarray()

In [ ]:
# split dataset for training and testing
#x_train,x_test,y_train,y_test= train_test_split(df['Text'],df['Label'],test_size=0.25)

In [ ]:
df['Label'] = df['Label'] - 1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(ngram_range=(1,1), max_features=5000)
train_features = tv.fit_transform(train_df['Text'])
test_features = tv.transform(test_df['Text'])

In [ ]:
# split dataset for training and testing
x_train,x_test,y_train,y_test= train_test_split(df['Text'],df['Label'],test_size=0.25)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
import tensorflow as tf

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np

# Tokenize the documents for Word2Vec
tokenized_documents_train = [word_tokenize(doc.lower()) for doc in x_train]
tokenized_documents_test = [word_tokenize(doc.lower()) for doc in x_test]

# Train the Word2Vec model using skip-gram
word2vec_model = Word2Vec(sentences=tokenized_documents_train, vector_size=100, window=5, sg=1, seed=42)

# Generate the feature matrix for training set using Word2Vec
word2vec_train_feature_matrix = []
for doc in tokenized_documents_train:
    doc_vector = np.mean([word2vec_model.wv[word] for word in doc if word in word2vec_model.wv], axis=0)
    word2vec_train_feature_matrix.append(doc_vector)

# Generate the feature matrix for testing set using Word2Vec
word2vec_test_feature_matrix = []
for doc in tokenized_documents_test:
    doc_vector = np.mean([word2vec_model.wv[word] for word in doc if word in word2vec_model.wv], axis=0)
    word2vec_test_feature_matrix.append(doc_vector)

# Convert the feature matrices to NumPy arrays
word2vec_train_feature_matrix = np.array(word2vec_train_feature_matrix)
word2vec_test_feature_matrix = np.array(word2vec_test_feature_matrix)

# Step 3: Use each feature matrix with a deep learning model (Simple CNN)
# Train and evaluate the CNN model using Word2Vec features
def create_cnn_model_word2vec(input_dim):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=100, input_length=input_dim))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the CNN model using Word2Vec features
cnn_model_word2vec = create_cnn_model_word2vec(word2vec_train_feature_matrix.shape[1])
cnn_model_word2vec.fit(word2vec_train_feature_matrix, y_train, epochs=5, batch_size=32, validation_split=0.2)
cnn_word2vec_probs = cnn_model_word2vec.predict(word2vec_test_feature_matrix)
#cnn_word2vec_preds = cnn_model_word2vec.predict_classes(word2vec_test_feature_matrix)
cnn_word2vec_preds = (cnn_word2vec_probs > 0.5).astype(int)

# Evaluate the model performance
accuracy_word2vec = accuracy_score(y_test, cnn_word2vec_preds)
conf_matrix_word2vec = confusion_matrix(y_test, cnn_word2vec_preds)
classification_report_word2vec = classification_report(y_test, cnn_word2vec_preds, target_names=['Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5'])


print("\nCNN Model with Word2Vec Features:")
print("Testing Accuracy: {:.4f}".format(accuracy_word2vec))
print("\nConfusion Matrix:")
print(conf_matrix_word2vec)
print("\nClassification Report:")
print(classification_report_word2vec)


Epoch 1/5
271/271 [==============================] - 3s 6ms/step - loss: -9770.2422 - accuracy: 0.1042 - val_loss: -53499.9102 - val_accuracy: 0.0988
Epoch 2/5
271/271 [==============================] - 2s 6ms/step - loss: -384197.5000 - accuracy: 0.1038 - val_loss: -1034225.4375 - val_accuracy: 0.0988
Epoch 3/5
271/271 [==============================] - 1s 6ms/step - loss: -2794849.2500 - accuracy: 0.1038 - val_loss: -5344282.0000 - val_accuracy: 0.0988
Epoch 4/5
271/271 [==============================] - 1s 5ms/step - loss: -10099826.0000 - accuracy: 0.1038 - val_loss: -16196393.0000 - val_accuracy: 0.0988
Epoch 5/5
113/113 [==============================] - 0s 2ms/step

CNN Model with Word2Vec Features:
Testing Accuracy: 0.1055

Confusion Matrix:
[[   0  805    0    0    0]
 [   0  381    0    0    0]
 [   0  459    0    0    0]
 [   0  774    0    0    0]
 [   0 1191    0    0    0]]

Classification Report:
              precision    recall  f1-score   support

     Class 1       0

In [ ]:
# Step 1: Apply the word uni-gram method and compute feature matrix
vectorizer = CountVectorizer()
feature_matrix_unigram = vectorizer.fit_transform(df['Text'])

# Step 2: Apply Word2Vec embedding model
tokenized_documents = [word_tokenize(doc.lower()) for doc in df['Text']]
word2vec_model = Word2Vec(sentences=tokenized_documents, vector_size=100, window=5, sg=1, seed=42)

word2vec_feature_matrix = []
for doc in tokenized_documents:
    doc_vector = np.mean([word2vec_model.wv[word] for word in doc if word in word2vec_model.wv], axis=0)
    word2vec_feature_matrix.append(doc_vector)

word2vec_feature_matrix = np.array(word2vec_feature_matrix)

# Split the dataset into training and testing sets
X_train_unigram, X_test_unigram, y_train, y_test = train_test_split(feature_matrix_unigram, df['Label'], test_size=0.25, random_state=42)
X_train_word2vec, X_test_word2vec = train_test_split(word2vec_feature_matrix, test_size=0.25, random_state=42)

In [ ]:
X_train_unigram_dense = X_train_unigram.toarray()
X_train_word2vec_dense = X_train_word2vec

In [ ]:
from keras.models import Sequential, Model
from keras.layers import InputLayer, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Define CNN model for unigram features
def create_cnn_model_unigram(input_dim):
    model = Sequential()
    model.add(InputLayer(input_shape=(input_dim,)))
    model.add(Embedding(input_dim=input_dim, output_dim=100))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define CNN model for Word2Vec features
def create_cnn_model_word2vec(input_dim):
    model = Sequential()
    model.add(InputLayer(input_shape=(input_dim,)))
    model.add(Embedding(input_dim=input_dim, output_dim=100))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
# Train and evaluate the CNN model using unigram features
cnn_model_unigram = create_cnn_model_unigram(X_train_unigram_dense.shape[1])
cnn_model_unigram.fit(X_train_unigram_dense, y_train, epochs=5, batch_size=32)
cnn_unigram_probs = cnn_model_unigram.predict(X_test_unigram.toarray())
cnn_unigram_preds = (cnn_unigram_probs > 0.5).astype(int)

# Train and evaluate the CNN model using Word2Vec features
cnn_model_word2vec = create_cnn_model_word2vec(X_train_word2vec_dense.shape[1])
cnn_model_word2vec.fit(X_train_word2vec_dense, y_train, epochs=5, batch_size=32)
cnn_word2vec_probs = cnn_model_word2vec.predict(X_test_word2vec)
cnn_word2vec_preds = (cnn_word2vec_probs > 0.5).astype(int)


Epoch 1/5
339/339 [==============================] - 100s 255ms/step - loss: -70032.7812 - accuracy: 0.1031
Epoch 2/5
339/339 [==============================] - 85s 250ms/step - loss: -2991295.7500 - accuracy: 0.1028
Epoch 3/5
339/339 [==============================] - 84s 249ms/step - loss: -21314934.0000 - accuracy: 0.1028
Epoch 4/5
339/339 [==============================] - 84s 248ms/step - loss: -74962056.0000 - accuracy: 0.1028
Epoch 5/5
113/113 [==============================] - 8s 66ms/step
Epoch 1/5
339/339 [==============================] - 3s 5ms/step - loss: -30694.4727 - accuracy: 0.1028
Epoch 2/5
339/339 [==============================] - 2s 5ms/step - loss: -1212560.3750 - accuracy: 0.1028
Epoch 3/5
339/339 [==============================] - 2s 5ms/step - loss: -8547040.0000 - accuracy: 0.1028
Epoch 4/5
339/339 [==============================] - 2s 5ms/step - loss: -30491032.0000 - accuracy: 0.1028
Epoch 5/5
113/113 [==============================] - 0s 3ms/step


In [ ]:
print("\nCNN Model with Word2Vec Features:")
print("Testing Accuracy: {:.4f}".format(accuracy_word2vec))
print("\nConfusion Matrix:")
print(conf_matrix_word2vec)
print("\nClassification Report:")
print(classification_report_word2vec)


CNN Model with Word2Vec Features:


NameError: ignored

In [ ]:
 nltk.download('punkt')

# **Word2Vec Embedding**

# **Convolutional Neural Network (CNN)**